In [19]:
import copy

class Node:
    def __init__(self, value, depth, available, might_available):
        self.value = value
        self.depth = depth
        self.available = available
        # dictionary to store partial_available nodes
        self.might_available = might_available
        
        # create dict instance for all bindings
        self.bindings = [ [] for _ in range(v_num-1)]        # or can call it ancestors at different levels
        # they represent what to substract from might_available
        
    def merge_available(self, other):
        '''
        for i in other:
            if i not in self.available:
                self.available.append(i)
        '''
        # increment values from uniquely other list
        for i in other:
            if i not in self.available:
                if i in self.might_available:
                    # increment partial_available value
                    self.might_available[i] = self.might_available[i]+1
                else:
                    self.might_available[i] = 1
                    
        
        # intersection of two lists
        tmp = copy.copy(self.available)
        for i in self.available:
            if i not in other:
                
                tmp.remove(i)
                '''
                if i == '57':
                    print('catch 57')
                    print("depth is")
                    print(self.depth)
                '''
                if i in self.might_available:
                    # increment partial_available value
                    self.might_available[i] = self.might_available[i]+1
                else:
                    self.might_available[i] = 1
        self.available = tmp
        
    # choose the maximum for each value
    def merge_might_available(self, other):
        for i in other.keys():
            if i in self.might_available.keys():
                self.might_available[i] = max(other[i], self.might_available[i])
            else:
                self.might_available[i] = other[i]
        for i in self.might_available.keys():
            if i in other.keys():
                self.might_available[i] = max(other[i], self.might_available[i])          
    
    def merge_bindings(self, other_node):
        other_bindings = other_node.bindings
        for i in range(v_num-1):
            for j in other_bindings[i]:
                if j not in self.bindings[i]:
                    self.bindings[i].append(j)
        
        
    def __str__(self):
        ret="value is:"+repr(self.value)+"    depth is:"+repr(self.depth)
        return ret
        
def readFile(filename):
    file_tmp = open(filename, 'r')
    lines = file_tmp.readlines()
    v = int(lines[3].split(' ')[2])
    lines = lines[6:]
    lines.pop()
    lines.pop()
    return v,lines

def buildAjacencyList(lines, v_num):
    v_dictionary = {}
    for line in lines:
        line = line.strip()
        splitted = line.split(' ')
        #print(splitted)
        if splitted[0] in v_dictionary.keys():
            v_dictionary[splitted[0]].append(splitted[1])
        else:
            v_dictionary[splitted[0]] = []
            (v_dictionary[splitted[0]]).append(splitted[1])
            
        if splitted[1] in v_dictionary.keys():    
            v_dictionary[splitted[1]].append(splitted[0])
        else:
            v_dictionary[splitted[1]] = []
            (v_dictionary[splitted[1]]).append(splitted[0])
    return v_dictionary

def print_matrix_list(list):
    for i in list:
        #print("------------------------------")
        if i != None:
            print(i)
            print(i.available)
            print(i.might_available)
            #print(i.bindings)
            

def algorithm(v_dictionary, starting_vertex, v_num):
    
    
    
    # create node for starting vertex
    #get all keys
    all_values = []
    for i in v_dictionary.keys():
        all_values.append(i)
        
    temp_avai = copy.copy(all_values)
    temp_avai.remove(starting_vertex)
    # create a starting dictionary of nothing because there is no underterministic node
    temp_might_avai = {}
    
    starting_v = Node(starting_vertex,0,temp_avai,temp_might_avai)
    #print(starting_v.available)
    # create (v_num-1)*(v_num) matrix to store nodes
    
    matrix = [ [None for _ in range(v_num) ] for _ in range(v_num-1)]      
    # excluding starting vertex because apprantly every graph starts with node 1
    #print(len(matrix))
    #matrix[1][1] = 10
    #print(matrix)
    # doing the starting vertex
    #print(v_dictionary[starting_vertex])
    for n in v_dictionary[starting_vertex]:
        #print(n)
        temp_avai = copy.copy(starting_v.available)
        temp_avai.remove(n)
        # again, no underterminstic value
        matrix[0][int(n)-1] = Node(n,1,temp_avai, temp_might_avai)
        #print(matrix[0][int(n)-1].available)
    #print(matrix)
    
    # do the remaining levels
    for i in range(0,v_num-2):
        for j in range(0,v_num):
            if matrix[i][j] != None:
                #print(i)
                #print(j)
                node = matrix[i][j]
                #print(node)
                for n in v_dictionary[node.value]:
                    # it must be available so do a transfer from parent to child
                    if n in node.available:
                        temp_avai = copy.copy(node.available)
                        temp_avai.remove(n)
                        temp_might_avai = copy.deepcopy(node.might_available)
                        #print(matrix[i+1][int(n)-1])
                        # simply do a deterministic transfer if permitted
                        if matrix[i+1][int(n)-1] == None:
                            #print("node create")
                            #print(int(n))
                            #print("depth is")
                            #print(i+2)
                            matrix[i+1][int(n)-1] = Node(n,i+2,temp_avai, temp_might_avai)
                            matrix[i+1][int(n)-1].bindings[i].append(node.value)
                        
                        # if it already existed, do a intersection of available nodes and 
                        # put everything else in might_available list
                        # also merge might_available list
                        else:
                            '''
                            print("node merge")
                            #print(i)
                            #print(matrix[i+1][int(n)-1])
                            print(matrix[i+1][int(n)-1])
                            print("node available list")
                            print(node.available)
                            print("existing list")
                            print(matrix[i+1][int(n)-1].available)
                            print("after merging list")
                            '''
                            matrix[i+1][int(n)-1].merge_available(temp_avai)
                            #print(matrix[i+1][int(n)-1].available)
                            matrix[i+1][int(n)-1].merge_might_available(temp_might_avai)
                            matrix[i+1][int(n)-1].bindings[i].append(node.value)
                            matrix[i+1][int(n)-1].merge_bindings(node)
                    
                    # if it is potentially available, check whether all values in might_avai >=1
                    # yes then decrement by 1 and add path, otherwise no
                    elif n in node.might_available:
                        
                        gr_or_eq_one_elements = []
                        '''
                        for k in node.might_available.keys():
                            if node.might_available[k]<1:
                                gr_or_eq_one = False
                                break
                        
                        if node.might_available[n]<1:
                            gr_or_eq_one = False
                        '''
                        for b in range(v_num-1):
                            #print("aaaaa")
                            if n in node.bindings[b]:
                                for k in node.bindings[b]:
                                    if k in node.might_available and node.might_available[k]>=1:
                                        for e in node.bindings[b]:
                                            #if e not in gr_or_eq_one_elements:
                                            gr_or_eq_one_elements.append(e)
                                        break
                                    
                        if len(gr_or_eq_one_elements)>0:       #node.might_available[n]>=1:    
                            temp_avai = copy.copy(node.available)
                            temp_might_avai = copy.deepcopy(node.might_available)
                            del temp_might_avai[n]
                            '''
                            for b in range(v_num-1):
                                if n in node.bindings[b]:                            
                                    for k in temp_might_avai.keys():
                                        temp_might_avai[k] = temp_might_avai[k]-1
                            '''
                            for e in gr_or_eq_one_elements:
                                if e in temp_might_avai:
                                    temp_might_avai[e] = temp_might_avai[e]-1
                            
                            #temp_might_avai[n] = temp_might_avai[n]-1
                                
                            if matrix[i+1][int(n)-1] == None:
                                #print("node create")
                                #print(int(n)-1)
                                matrix[i+1][int(n)-1] = Node(n,i+2,temp_avai, temp_might_avai)
                                matrix[i+1][int(n)-1].bindings[i].append(node.value)
                            else:
                                matrix[i+1][int(n)-1].merge_available(temp_avai)
                                matrix[i+1][int(n)-1].merge_might_available(temp_might_avai)
                                matrix[i+1][int(n)-1].bindings[i].append(node.value)
                                matrix[i+1][int(n)-1].merge_bindings(node)
                            
                            
                    
        #print_matrix_list(matrix[i+1])
    return matrix


# small test cases works, but larger ones still not
path = './FHCPCS'
v_num, lines = readFile(path+'/graph0.hcp')    
v_dictionary = buildAjacencyList(lines, v_num)

result = algorithm(v_dictionary, '1', v_num)

#print(v_dictionary.keys())
#print(v_num)
#print(result)
for i in range(0,len(result)):
    print_matrix_list(result[i])
#print_matrix_list(result[0])
#print(result[0][2].available)

value is:'2'    depth is:1
['3', '4', '5', '7', '6']
{}
value is:'3'    depth is:1
['2', '4', '5', '7', '6']
{}
value is:'4'    depth is:1
['2', '3', '5', '7', '6']
{}
value is:'2'    depth is:2
['3', '5', '7', '6']
{}
value is:'4'    depth is:2
['3', '5', '7', '6']
{}
value is:'5'    depth is:2
['7', '6']
{'2': 2, '3': 2, '4': 1}
value is:'7'    depth is:2
['2', '3', '5', '6']
{}
value is:'2'    depth is:3
['7', '6']
{'3': 1, '4': 0}
value is:'3'    depth is:3
['7', '6']
{'2': 1, '4': 0}
value is:'4'    depth is:3
['7', '6']
{'2': 1, '3': 1}
value is:'5'    depth is:3
['3', '7', '6']
{}
value is:'6'    depth is:3
[]
{'2': 3, '3': 3, '4': 1, '5': 1, '7': 1}
value is:'7'    depth is:3
['3', '5', '6']
{}
value is:'3'    depth is:4
['7', '6']
{}
value is:'5'    depth is:4
[]
{'2': 3, '3': 3, '4': 1, '7': 0}
value is:'6'    depth is:4
['3']
{'5': 1, '7': 1}
value is:'7'    depth is:4
[]
{'2': 3, '3': 3, '6': 1, '4': 1, '5': 0}
value is:'4'    depth is:5
[]
{'2': 3, '3': 3, '6': 0, '5': 0}


In [2]:
print(v_dictionary)

{'1': ['3', '9', '61'], '3': ['1', '40', '46'], '9': ['1', '25', '35'], '61': ['1', '14', '49'], '2': ['17', '31', '51'], '17': ['2', '21', '37'], '31': ['2', '7', '29'], '51': ['2', '13', '48'], '40': ['3', '6', '59'], '46': ['3', '25', '62'], '4': ['42', '47', '63'], '42': ['4', '41', '55'], '47': ['4', '12', '50'], '63': ['4', '8', '15'], '5': ['8', '18', '39'], '8': ['5', '50', '63'], '18': ['5', '15', '23'], '39': ['5', '34', '49'], '6': ['30', '40', '62'], '30': ['6', '54', '56'], '62': ['6', '24', '46'], '7': ['13', '31', '58'], '13': ['7', '51', '66'], '58': ['7', '41', '64'], '50': ['8', '34', '47'], '25': ['9', '36', '46'], '35': ['9', '14', '32'], '10': ['16', '27', '44'], '16': ['10', '52', '53'], '27': ['10', '60', '65'], '44': ['10', '22', '28'], '11': ['19', '45', '54'], '19': ['11', '52', '56'], '45': ['11', '28', '38'], '54': ['11', '30', '59'], '12': ['21', '47', '55'], '21': ['12', '17', '34'], '55': ['12', '37', '42'], '66': ['13', '33', '64'], '14': ['35', '57', '6

In [4]:
'''
debugging with remove
note: we should never removing when iterating through a list, a python problem
'''

#node merge
#value is:'20'    depth is:4
node_available_list=['3', '9', '2', '17', '31', '51', '40', '46', '4', '42', '47', '63', '5', '8', '18', '39', '6', '30', '62', '7', '13', '58', '50', '25', '35', '10', '16', '27', '44', '11', '19', '45', '54', '12', '21', '55', '66', '15', '20', '52', '53', '37', '23', '56', '32', '34', '22', '38', '60', '49', '24', '36', '26', '43', '64', '65', '28', '29', '41', '33', '48', '59']
existing_list=['3', '61', '2', '17', '31', '51', '40', '46', '4', '42', '47', '63', '5', '8', '18', '39', '6', '30', '62', '7', '13', '58', '50', '25', '10', '16', '27', '44', '11', '19', '45', '54', '12', '21', '55', '66', '14', '57', '15', '52', '53', '37', '23', '56', '34', '22', '38', '60', '49', '24', '36', '26', '43', '64', '65', '28', '29', '41', '33', '48', '59']
after_merging_list=['3', '2', '17', '31', '51', '40', '46', '4', '42', '47', '63', '5', '8', '18', '39', '6', '30', '62', '7', '13', '58', '50', '25', '10', '16', '27', '44', '11', '19', '45', '54', '12', '21', '55', '66', '57', '15', '52', '53', '37', '23', '56', '34', '22', '38', '60', '49', '24', '36', '26', '43', '64', '65', '28', '29', '41', '33', '48', '59']

node_available_list.sort()
print(node_available_list)
existing_list.sort()
print(existing_list)
after_merging_list.sort()
print(after_merging_list)

for i in node_available_list:
    if i not in existing_list:
        node_available_list.remove(i)
print(node_available_list)

['10', '11', '12', '13', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '58', '59', '6', '60', '62', '63', '64', '65', '66', '7', '8', '9']
['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '33', '34', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '7', '8']
['10', '11', '12', '13', '15', '16', '17', '18', '19', '2', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '33', '34', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '5

## Discussion
#### This approach still does not work, but I think there is some value in it. It partially solves the problem proposed in the previous modification. Namely, it will not repeatedly go to the same nodes in small test cases. However, in large datasets, it still shows all paths in the lower levels of the holograph. I will investigate it further and try to figure out why it happens. 